In [1]:
import pronto
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from graphviz import Digraph
from IPython.display import Image

In [2]:
#ont = pronto.Ontology('hp.obo')
ont = pronto.Ontology('https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo')

<ipython-input-2-11e910691231>:2: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  ont = pronto.Ontology('https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo')


In [3]:
import os
os.getcwd()

'C:\\Users\\StatisticalGenetics\\git\\Ontology'

In [4]:
# HP:0001249, HP:0001250

target_HP = 'HP:0001249'

superclass

In [5]:
# superclassタームを抽出
sup_target_HP_list = list(ont[target_HP].superclasses()) # 自分（HP:0001249）を入れる場合は引数不要（デフォルト）
sup_target_HP_list

[Term('HP:0001249', name='Intellectual disability'),
 Term('HP:0011446', name='Abnormality of higher mental function'),
 Term('HP:0012759', name='Neurodevelopmental abnormality'),
 Term('HP:0012638', name='Abnormal nervous system physiology'),
 Term('HP:0000707', name='Abnormality of the nervous system'),
 Term('HP:0000118', name='Phenotypic abnormality'),
 Term('HP:0000001', name='All')]

In [6]:
target_HP_name = sup_target_HP_list[0].name

In [7]:
# 抽出したタームを階層ごとに分ける

# まず全階層の箱superclass_layer_listと、その中に階層ごとの箱superclass_●layer_listを作っておく


superclass_layer_list =[]

for i in range(1,10000):
    exec("superclass_"+str(i)+"layer_list" + " = list(ont[target_HP].superclasses(distance="+str(i)+"))")
    if list(ont[target_HP].superclasses(distance=i-1)) == list(ont[target_HP].superclasses(distance=i)):
        print("Max depth:" + str(i-1))         
        break;
        
    superclass_layer_list.append("superclass_"+str(i)+"layer_list")

max_len=i

Max depth:5


In [8]:
# 1つ前のlayerのものをひいていく その階層のタームだけが残る
for i in range(max_len-2,0,-1):
    print(i)
    exec("superclass_"+str(i+1)+"layer_list" "= set("+superclass_layer_list[i]+") - set(" + superclass_layer_list[i-1]+ ")")

4
3
2
1


In [9]:
target_HP_rep=target_HP.replace(':', '')

In [10]:
# 階層ごとの箱subclass_●layer_listにタームを振り分ける

for i,superclasslist in enumerate(superclass_layer_list):
    exec("superclass_"+str(i+1)+"layer_list_id = eval(superclasslist)")
    exec("superclass_"+str(i+1)+"layer_list_id = [s.id + s.name for s in superclass_"+ str(i+1) +"layer_list_id]")
    exec("superclass_"+str(i+1)+"layer_list_id = [s.replace(':', '') for s in superclass_" +str(i+1)+"layer_list_id]")
    exec("superclass_"+str(i+1)+"layer_list_id = list(reversed(superclass_"+str(i+1)+"layer_list_id))")

# enumerate()関数の引数にリストなどのイテラブルオブジェクトを指定する
# インデックス番号, 要素の順に取得できる

In [11]:
from graphviz import Digraph
import networkx as nx

G = Digraph(format="png")

In [12]:
# 関数の定義 subclassでのノードの設定

def connected_judge(HP_1,HP_2,replaced=True,name_plus=True):
    # HP_1 parentid
    # HP_2 parentid
    if replaced:
        HP_1 = HP_1.replace('HP', 'HP:')
        HP_2 = HP_2.replace('HP', 'HP:')
    if name_plus:
        HP_1 = HP_1[:10]
        HP_2 = HP_2[:10]
    HP_2_list = list(ont[HP_2].superclasses(distance=1))
    HP_2_list = [s.id for s in HP_2_list]
    return HP_1 in HP_2_list

In [13]:
# superclassでのエッジの設定

# subclass_1layer_listとそれ以外に分けて処理

for superclasslist in superclass_layer_list:
    tmplist = eval(superclasslist+"_id")
    if superclasslist == 'superclass_1layer_list':
        for hp_node_ind_1 in range(len(tmplist)-1):
            G.edge(tmplist[hp_node_ind_1][9:],target_HP_name)
            before_list = copy.copy(tmplist)            
    else:
        for hp_id_super_class in tmplist:
            for hp_id_sub_class_candidate in before_list:
                if connected_judge(hp_id_super_class,hp_id_sub_class_candidate,replaced=True):
                    G.edge(hp_id_super_class[9:],hp_id_sub_class_candidate[9:])
        before_list = copy.copy(tmplist)

In [14]:
G.node(target_HP_name, shape="ellipse", style="filled", fillcolor="lightblue2")

In [15]:
G.render()
G.view()
#!dot -T png Digraph.gv > test.png
#Image('test.png')

'Digraph.gv.png'

In [16]:
# HP:0001249の場合
# HP:0012638の中に、HP0012759とHP0011446のchildnodeが２つあることを確認

list(ont["HP:0012638"].subclasses(distance=1))

[Term('HP:0012638', name='Abnormal nervous system physiology'),
 Term('HP:0000223', name='Abnormality of taste sensation'),
 Term('HP:0000708', name='Behavioral abnormality'),
 Term('HP:0001250', name='Seizure'),
 Term('HP:0001283', name='Bulbar palsy'),
 Term('HP:0001298', name='Encephalopathy'),
 Term('HP:0001311', name='Abnormal nervous system electrophysiology'),
 Term('HP:0002015', name='Dysphagia'),
 Term('HP:0002200', name='Pseudobulbar signs'),
 Term('HP:0002315', name='Headache'),
 Term('HP:0002483', name='Bulbar signs'),
 Term('HP:0003388', name='Easy fatigability'),
 Term('HP:0003472', name='Hypocalcemic tetany'),
 Term('HP:0004408', name='Abnormality of the sense of smell'),
 Term('HP:0011442', name='Abnormal central motor function'),
 Term('HP:0011446', name='Abnormality of higher mental function'),
 Term('HP:0011730', name='Abnormal central sensory function'),
 Term('HP:0012285', name='Abnormal hypothalamus physiology'),
 Term('HP:0012535', name='Abnormal synaptic transmi

subclass

In [17]:
# subclassタームを抽出
sub_target_HP_list = list(ont[target_HP].subclasses()) 

In [18]:
# 抽出したタームを階層ごとに分ける

# まず全階層の箱subclass_layer_listと、その中に階層ごとの箱subclass_●layer_listを作っておく

subclass_layer_list =[]

for i in range(1,10000):
    exec("subclass_"+str(i)+"layer_list" + " = list(ont[target_HP].subclasses(distance="+str(i)+"))")
    if list(ont[target_HP].subclasses(distance=i-1)) == list(ont[target_HP].subclasses(distance=i)):
        print("Max depth:" + str(i-1))         
        break;
        
    subclass_layer_list.append("subclass_"+str(i)+"layer_list")

max_len=i

Max depth:1


In [19]:
# 1つ前のlayerのものをひいていく その階層のタームだけが残る
for i in range(max_len-2,0,-1):
    print(i)
    exec("subclass_"+str(i+1)+"layer_list" "= set("+subclass_layer_list[i]+") - set(" + subclass_layer_list[i-1]+ ")")

In [20]:
# 階層ごとの箱subclass_●layer_listにタームを振り分ける

for i,subclasslist in enumerate(subclass_layer_list):
    exec("subclass_"+str(i+1)+"layer_list_id = eval(subclasslist)")
    exec("subclass_"+str(i+1)+"layer_list_id = [s.id + s.name for s in subclass_"+ str(i+1) +"layer_list_id]")
    exec("subclass_"+str(i+1)+"layer_list_id = [s.replace(':', '') for s in subclass_" +str(i+1)+"layer_list_id]")
    exec("subclass_"+str(i+1)+"layer_list_id = list(reversed(subclass_"+str(i+1)+"layer_list_id))")

# enumerate()関数の引数にリストなどのイテラブルオブジェクトを指定する
# インデックス番号, 要素の順に取得できる

In [21]:
subclass_1layer_list

[Term('HP:0001249', name='Intellectual disability'),
 Term('HP:0001256', name='Intellectual disability, mild'),
 Term('HP:0002187', name='Intellectual disability, profound'),
 Term('HP:0002342', name='Intellectual disability, moderate'),
 Term('HP:0006887', name='Intellectual disability, progressive'),
 Term('HP:0006889', name='Intellectual disability, borderline'),
 Term('HP:0010864', name='Intellectual disability, severe')]

In [22]:
# 関数の定義 subclassでのノードの設定

def connected_judge(HP_1,HP_2,replaced=True,name_plus=True):
    # HP_1 parentid
    # HP_2 parentid
    if replaced:
        HP_1 = HP_1.replace('HP', 'HP:')
        HP_2 = HP_2.replace('HP', 'HP:')
    if name_plus:
        HP_1 = HP_1[:10]
        HP_2 = HP_2[:10]
    HP_2_list = list(ont[HP_2].subclasses(distance=1))
    HP_2_list = [s.id for s in HP_2_list]
    return HP_1 in HP_2_list

In [23]:
# subclassでのエッジの設定

# subclass_1layer_listとそれ以外に分けて処理

for subclasslist in subclass_layer_list:
    tmplist = eval(subclasslist+"_id")
    if subclasslist == 'subclass_1layer_list':
        for hp_node_ind_1 in range(len(tmplist)-1):
            G.edge(target_HP_name, tmplist[hp_node_ind_1][9:])
            before_list = copy.copy(tmplist)
            
    else:
        for hp_id_sub_class in tmplist:
            for hp_id_sub_class_candidate in before_list:
                if connected_judge(hp_id_sub_class, hp_id_sub_class_candidate, replaced=True):
                    G.edge(hp_id_sub_class_candidate[9:], hp_id_sub_class[9:])
        before_list = copy.copy(tmplist)   

In [24]:
# tmplist

In [25]:
G.render()
G.view()
#!dot -T png Digraph.gv > test.png
#Image('test.png')

'Digraph.gv.png'

In [ ]:
!git add .

In [ ]:
!git commit

In [ ]:
!git commit -m "recommit" -v

In [ ]:
!git push